In [1]:
from tensorflow.keras.models import load_model
import os
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
import imutils
import cv2

import socket
import pickle
import base64
import json
import time

#load the model
trainedModel = load_model("multiLabel_model_true_041219.h5")

base_dir_path = os.getcwd() # current directory
train_dir_path = os.path.join(base_dir_path,'Train') #train directory
#generate dictionnary of classes
AllClassNames = os.listdir(train_dir_path)
num_of_classes = len(AllClassNames)
DictOfClasses = {i : AllClassNames[i] for i in range(0, len(AllClassNames))}

def predict_fruit(imgPath, Model, dicOfClasses):
    img = cv2.imread(imgPath)
    output = imutils.resize(img, width=600)
    # pre-process the image for classification
    img = cv2.resize(img, (160, 160))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    pImg = preprocess_input(img)
    
    #predict classes
    prob = Model.predict(pImg)[0]
    top4 = np.argsort(prob)[::-1][:4]
    
    # loop over the indexes of the high confidence class labels
    for (i, j) in enumerate(top4):
        # build the label and draw the label on the image
        label = "{}: {:.2f}%".format(DictOfClasses[j], prob[j] * 100)
        cv2.putText(output, label, (10, (i * 30) + 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0 ,255), 2)
    
    result = "result.jpg"
    cv2.imwrite(result , output)
    
    return result


In [ ]:
def genJson(resultImg):
    
    result = predict_fruit(resultImg, trainedModel, DictOfClasses)
    
    with open(result, "rb") as imageFile:
        str = base64.b64encode(imageFile.read())
    im = str.decode('utf-8')
    m ={"id": 2, "image": im}
    jsonObj = json.dumps(m)
    msg = jsonObj.encode()
    return msg


#######################################################################################
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind(('localhost', 8080))
soc.listen(1)
rs = ""
while True:
    print('a l\'ecoute')
    (socClient, addrClient) = soc.accept()
    print('address du client', addrClient)
    try:
        while True:
            
            data = socClient.recv(8192 * 32)
            rep =data.decode('utf-8') 
            if data == '' or len(data) == 0:
                break
            jsr = json.loads(rep)
            imgdata = base64.b64decode(jsr['image'])
            resultImg = "response.jpg"
            with open(resultImg, 'wb') as f:
                f.write(imgdata)
            msg = genJson(resultImg)
            socClient.sendall(msg)
        socClient.close()
    except socket.error:
        print('client interrompu')
soc.close()

a l'ecoute
address du client ('127.0.0.1', 55991)
a l'ecoute
address du client ('127.0.0.1', 56028)
a l'ecoute
address du client ('127.0.0.1', 56102)
a l'ecoute
address du client ('127.0.0.1', 56123)
a l'ecoute
